In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [4]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD             CLIRAD `coolr_bands`           
            layer      coolrg                layer     coolrg
pressure                                                     
0.000312        1  115.678140                    1  74.628977
0.000750        2   49.945176                    2  56.831390
0.001052        3   39.870909                    3  49.200647
0.001476        4   31.496472                    4  40.196935
0.002070        5   24.905950                    5  30.703120
0.002904        6   19.856896                    6  21.966768
0.004074        7   16.013594                    7  15.319263
0.005714        8   13.056533                    8  11.270433
0.008015        9   10.713331                    9   9.283729
0.011243       10    8.768981                   10   8.282397
0.015771       11    7.068433                   11   7.373060
0.022122       12    5.504420                   12   6.211677
0.031031       13    3.931102                   13   4.606832
0.043528       14    1.993928                   14   2.373989
0.061057       15    1.344404                   15   1.273046
0.085645       16    2.366248                   16   1.906097
0.120136       17    3.454192                   17   2.834941
0.168516       18    4.304741                   18   3.766175
0.236378       19    5.217085                   19   5.051198
0.331549       20    6.320256                   20   6.660456
0.465100       21    7.883281                   21   8.523439
0.652400       22   10.173709                   22  10.521826
0.915100       23   10.297246                   23   9.984425
1.283650       24    8.669062                   24   8.019326
1.800600       25    7.333199                   25   7.209383
2.525700       26    5.930090                   26   6.341475
3.542800       27    4.887613                   27   5.243039
4.969550       28    4.110024                   28   4.030960
6.970850       29    3.445123                   29   3.373962
9.778100       30    2.838566                   30   3.166106
13.715850      31    2.272536                   31   2.763069
19.239350      32    1.748551                   32   1.966048
26.987250      33    1.286887                   33   1.272803
37.855300      34    0.949779                   34   1.050163
53.100050      35    0.499832                   35   0.686584
73.887500      36   -0.030260                   36   0.017714
97.662500      37   -0.361817                   37  -0.448357
121.437500     38   -0.250642                   38  -0.318052
145.212500     39    0.028051                   39   0.014138
168.987500     40    0.077256                   40   0.057009
192.762500     41    0.096964                   41   0.068626
216.537500     42    0.104866                   42   0.077443
240.312500     43    0.106293                   43   0.085827
264.087500     44    0.103900                   44   0.091492
287.862500     45    0.099269                   45   0.093934
311.637500     46    0.093332                   46   0.092816
335.412500     47    0.086752                   47   0.087625
359.187500     48    0.079961                   48   0.081298
382.962500     49    0.073218                   49   0.072288
406.737500     50    0.066797                   50   0.063179
430.512500     51    0.060770                   51   0.054704
454.287500     52    0.055239                   52   0.045630
478.062500     53    0.050149                   53   0.039565
501.837500     54    0.045538                   54   0.032364
525.612500     55    0.041453                   55   0.028959
549.387500     56    0.037826                   56   0.024179
573.162500     57    0.034762                   57   0.022037
596.937500     58    0.032587                   58   0.020077
620.712500     59    0.031871                   59   0.020075
644.487500     60    0.034319                   60   0.023438
668.262500     61    0.041451                   61   0.031646
692.037500     62    0.050328                 

# Fluxes by Layer

CRD                           CLIRAD             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -16.240850   0.000000 -16.240850 -16.082083   0.000001   
0.000624    2     -16.241314   0.009020 -16.232294 -16.082613   0.006052   
0.000876    3     -16.241538   0.010732 -16.230806 -16.082835   0.007967   
0.001229    4     -16.241815   0.012675 -16.229140 -16.083120   0.010308   
0.001723    5     -16.242158   0.014865 -16.227294 -16.083485   0.013030   
0.002417    6     -16.242588   0.017341 -16.225246 -16.083955   0.016024   
0.003391    7     -16.243128   0.020171 -16.222956 -16.084563   0.019165   
0.004757    8     -16.243814   0.023448 -16.220366 -16.085353   0.022433   
0.006672    9     -16.244699   0.027296 -16.217403 -16.086388   0.026026   
0.009359    10    -16.245857   0.031863 -16.213994 -16.087760   0.030353   
0.013128    11    -16.247401   0.037322 -16.210079 -16.089603   0.035894   
0.018415    12    -16.249509   0.043857 -16.205652 -16.092142   0.043051   
0.025830    13    -16.252486   0.051669 -16.200817 -16.095726   0.052092   
0.036232    14    -16.256957   0.060984 -16.195973 -16.101054   0.063098   
0.050823    15    -16.264792   0.072266 -16.192526 -16.109974   0.076121   
0.071291    16    -16.277615   0.088348 -16.189267 -16.124035   0.093269   
0.100000    17    -16.295212   0.113993 -16.181220 -16.143059   0.118776   
0.140271    18    -16.317669   0.152927 -16.164742 -16.167092   0.156334   
0.196760    19    -16.345819   0.209882 -16.135937 -16.196845   0.211291   
0.275997    20    -16.379975   0.293007 -16.086968 -16.232292   0.294155   
0.387100    21    -16.418483   0.414695 -16.003788 -16.271087   0.420617   
0.543100    22    -16.451503   0.593393 -15.858110 -16.302785   0.609838   
0.761700    23    -16.442648   0.847983 -15.594665 -16.293149   0.872690   
1.068500    24    -16.374471   1.154036 -15.220435 -16.225745   1.168184   
1.498800    25    -16.261208   1.482652 -14.778555 -16.114243   1.465486   
2.102400    26    -16.098414   1.844188 -14.254226 -15.953041   1.819814   
2.949000    27    -15.896882   2.237360 -13.659522 -15.753180   2.255978   
4.136600    28    -15.657418   2.685483 -12.971935 -15.512691   2.753154   
5.802500    29    -15.373149   3.212277 -12.160872 -15.223328   3.259335   
8.139200    30    -15.036910   3.829643 -11.207266 -14.872511   3.842523   
11.417000   31    -14.641917   4.536806 -10.105112 -14.455924   4.655394   
16.014700   32    -14.182600   5.315180  -8.867420 -13.965827   5.670304   
22.464000   33    -13.654220   6.122633  -7.531587 -13.392902   6.599527   
31.510500   34    -13.040138   6.887608  -6.152530 -12.710308   7.281038   
44.200100   35    -12.218891   7.494043  -4.724848 -11.778507   7.927977   
62.000000   36    -11.291356   7.620417  -3.670938 -10.708345   8.305645   
85.775000   37    -10.914720   7.158561  -3.756159 -10.206471   7.853665   
109.550000  38    -11.594408   6.819257  -4.775150 -10.900093   7.284439   
133.325000  39    -12.916296   7.435259  -5.481037 -12.321087   7.809606   
157.100000  40    -14.242626   8.840591  -5.402035 -13.727780   9.256121   
180.875000  41    -15.557199  10.372741  -5.184458 -15.107391  10.796304   
204.650000  42    -16.854688  11.943312  -4.911376 -16.456513  12.338719   
228.425000  43    -18.131976  13.515936  -4.616040 -17.778018  13.878352   
252.200000  44    -19.386965  15.070281  -4.316685 -19.069876  15.411952   
275.975000  45    -20.618774  16.594706  -4.024068 -20.333580  16.933353   
299.750000  46    -21.827057  18.082561  -3.744496 -21.570078  18.434426   
323.525000  47    -23.011887  19.530245  -3.481642 -22.778016  19.903791   
347.300000  48    -24.173591  20.936271  -3.237321 -23.961480  21.334060   
371.075000  49    -25.312676  22.300550  -3.012126 -25.119220  22.720785   
394.850000  50    -26.429763  23.623841  -2.805922 -26.252693  24.057865 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')